In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install thop
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.6 MB/s eta 0:00:00


In [3]:
"""
GFLOPs and Parameters for YOLOv11n-ReID (Triplet Loss)
"""

import torch
import torch.nn as nn
from thop import profile, clever_format

# ==========================
# DEFINE MODEL
# ==========================
class YOLOv11ReIDModel(nn. Module):
    def __init__(self, embedding_dim=256):
        super().__init__()

        from ultralytics import YOLO
        yolo = YOLO("yolo11n-cls.pt")

        self.backbone = nn.Sequential(*list(yolo.model.model.children())[:-1])

        for i, child in enumerate(self. backbone.children()):
            if i < 6:
                for param in child.parameters():
                    param.requires_grad = False

        self.gap = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self._init_embedding_layer(embedding_dim)

    def _init_embedding_layer(self, embedding_dim):
        with torch.no_grad():
            dummy = torch.randn(1, 3, 224, 224)
            for layer in self.backbone:
                dummy = layer(dummy)

            if len(dummy.shape) == 4:
                dummy = self.gap(dummy)

            dummy = self.flatten(dummy)
            feature_dim = dummy. shape[1]

        self.embedding = nn.Sequential(
            nn.Linear(feature_dim, embedding_dim),
            nn.BatchNorm1d(embedding_dim),
        )

        print(f"Feature dim: {feature_dim} -> Embedding dim: {embedding_dim}")

    def forward(self, x):
        for layer in self.backbone:
            x = layer(x)

        if len(x.shape) == 4:
            x = self.gap(x)

        x = self. flatten(x)
        x = self. embedding(x)
        x = nn.functional.normalize(x, p=2, dim=1)

        return x


# ==========================
# GFLOPs AND PARAMETERS
# ==========================
def calculate_model_complexity():
    print("=" * 60)
    print("TÍNH GFLOPs VÀ PARAMETERS CHO YOLOv11n-ReID")
    print("=" * 60)

    # model
    EMBEDDING_DIM = 256
    IMGSZ = 224

    print("\n1.  Khởi tạo model...")
    model = YOLOv11ReIDModel(embedding_dim=EMBEDDING_DIM)
    model.eval()

    # Parameters
    total_params = sum(p.numel() for p in model. parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    print(f"\n2. Parameters:")
    print(f"   - Total Parameters: {total_params:,}")
    print(f"   - Total Parameters (M): {total_params / 1e6:.2f} M")
    print(f"   - Trainable Parameters: {trainable_params:,}")
    print(f"   - Trainable Parameters (M): {trainable_params / 1e6:.2f} M")

    # FLOPs calculation by thop
    print(f"\n3.  Tính FLOPs (input size: {IMGSZ}x{IMGSZ})...")
    input_tensor = torch.randn(1, 3, IMGSZ, IMGSZ)

    with torch.no_grad():
        flops, params = profile(model, inputs=(input_tensor,), verbose=False)

    flops_formatted, params_formatted = clever_format([flops, params], "%.2f")

    print(f"   - FLOPs: {flops_formatted}")
    print(f"   - GFLOPs: {flops / 1e9:.2f}")
    print(f"   - Parameters (verify): {params_formatted}")

    # MACs (Multiply-Accumulate Operations)
    macs = flops / 2  # 1 MAC = 2 FLOPs (1 multiply + 1 add)
    print(f"   - MACs: {macs / 1e9:.2f} G")

    # Model size (MB)
    model_size_mb = total_params * 4 / (1024 * 1024)  # float32 = 4 bytes
    print(f"\n4. Model Size:")
    print(f"   - Size (FP32): {model_size_mb:.2f} MB")
    print(f"   - Size (FP16): {model_size_mb / 2:.2f} MB")

    print("\n" + "=" * 60)
    print("KẾT QUẢ TỔNG HỢP")
    print("=" * 60)
    print(f"Model: YOLOv11n-ReID (Triplet Loss)")
    print(f"Input Size: {IMGSZ} x {IMGSZ}")
    print(f"Parameters: {total_params / 1e6:.2f} M")
    print(f"GFLOPs: {flops / 1e9:.2f}")
    print(f"Model Size (FP32): {model_size_mb:.2f} MB")
    print("=" * 60)

    return {
        'params_M': total_params / 1e6,
        'gflops': flops / 1e9,
        'model_size_mb': model_size_mb
    }


# ==========================
# LOAD CHECKPOINT AND VERIFY
# ==========================
def verify_with_checkpoint(checkpoint_path):
    print("\n" + "=" * 60)
    print("VERIFY VỚI CHECKPOINT ĐÃ TRAIN")
    print("=" * 60)

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location='cpu')
    print(f"\nCheckpoint keys: {checkpoint.keys()}")
    print(f"Epoch: {checkpoint. get('epoch', 'N/A')}")
    print(f"Loss: {checkpoint.get('loss', 'N/A')}")
    print(f"Embedding dim: {checkpoint. get('embedding_dim', 'N/A')}")

    # Params calculation from state_dict
    state_dict = checkpoint['model_state_dict']
    total_params_from_ckpt = sum(v.numel() for v in state_dict.values())
    print(f"\nParameters từ checkpoint: {total_params_from_ckpt:,}")
    print(f"Parameters từ checkpoint (M): {total_params_from_ckpt / 1e6:.2f} M")

    # Load into model and verify
    EMBEDDING_DIM = checkpoint.get('embedding_dim', 256)
    model = YOLOv11ReIDModel(embedding_dim=EMBEDDING_DIM)
    model.load_state_dict(state_dict)
    model.eval()

    # FLOPs Calculation
    input_tensor = torch. randn(1, 3, 224, 224)
    with torch. no_grad():
        flops, params = profile(model, inputs=(input_tensor,), verbose=False)

    print(f"\nVerify sau khi load checkpoint:")
    print(f"   - Parameters: {params / 1e6:.2f} M")
    print(f"   - GFLOPs: {flops / 1e9:.2f}")

    return flops / 1e9, params / 1e6


# ==========================
# MAIN
# ==========================
if __name__ == "__main__":
    results = calculate_model_complexity()


TÍNH GFLOPs VÀ PARAMETERS CHO YOLOv11n-ReID

1.  Khởi tạo model...
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Feature dim: 256 -> Embedding dim: 256

2. Parameters:
   - Total Parameters: 1,267,168
   - Total Parameters (M): 1.27 M
   - Trainable Parameters: 66,304
   - Trainable Parameters (M): 0.07 M

3.  Tính FLOPs (input size: 224x224)...
   - FLOPs: 182.85M
   - GFLOPs: 0.18
   - Parameters (verify): 1.27M
   - MACs: 0.09 G

4. Model Size:
   - Size (FP32): 4.83 MB
   - Size (FP16): 2.42 MB

KẾT QUẢ TỔNG HỢP
Model: YOLOv11n-ReID (Triplet Loss)
Input Size: 224 x 224
Parameters: 1.27 M
GFLOPs: 0.18
Model Size (FP32): 4.83 MB
